In [1]:
## This file implements neural networks and random forest on p002ypresabs_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 96.00% for over-sampling data and 92.5% after regularization.
## For random forest, the accuracy is 95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 95.71%.

In [37]:
from numpy.random import seed
seed(100)
np.random.seed(123)
import tensorflow
tensorflow.random.set_seed(123)

In [38]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p002ypresabs_qual.csv')
df.shape

(255, 1759)

In [39]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [40]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     1
12     0
13     0
14     0
15     0
16     0
17     0
18     1
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [41]:
df['pheno'].value_counts()

0    222
1     30
2      3
Name: pheno, dtype: int64

In [42]:
df.head()

,id,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,...,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,120337,1,1,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [43]:
df_clean = df.drop(columns=['id'])

In [44]:
df_clean.shape

(255, 1758)

In [45]:
df_clean.head()

,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATTTTCAAATCCCAT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,...,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9026,group_9123,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1757) (255,)


In [47]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 222), (1, 222), (2, 222)]


In [48]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [49]:
############# Fully-Connected Neural Network ################

In [50]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [69]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [70]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [71]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 379us/step - loss: 0.6695 - accuracy: 0.6545 - val_loss: 0.4739 - val_accuracy: 0.7300
Epoch 2/100
466/466 [==============================] - 0s 101us/step - loss: 0.4437 - accuracy: 0.8047 - val_loss: 0.4465 - val_accuracy: 0.7800
Epoch 3/100
466/466 [==============================] - 0s 109us/step - loss: 0.3919 - accuracy: 0.8283 - val_loss: 0.3657 - val_accuracy: 0.8200
Epoch 4/100
466/466 [==============================] - 0s 103us/step - loss: 0.3294 - accuracy: 0.8562 - val_loss: 0.3235 - val_accuracy: 0.8500
Epoch 5/100
466/466 [==============================] - 0s 110us/step - loss: 0.3091 - accuracy: 0.8691 - val_loss: 0.3577 - val_accuracy: 0.8550
Epoch 6/100
466/466 [==============================] - 0s 103us/step - loss: 0.2961 - accuracy: 0.8734 - val_loss: 0.2903 - val_accuracy: 0.8850
Epoch 7/100
466/466 [==============================] - 0s 102us/step - loss: 0.2859 

Epoch 57/100
466/466 [==============================] - 0s 105us/step - loss: 0.0254 - accuracy: 0.9957 - val_loss: 0.1260 - val_accuracy: 0.9500
Epoch 58/100
466/466 [==============================] - 0s 102us/step - loss: 0.0237 - accuracy: 0.9957 - val_loss: 0.1406 - val_accuracy: 0.9400
Epoch 59/100
466/466 [==============================] - 0s 89us/step - loss: 0.0312 - accuracy: 0.9914 - val_loss: 0.1267 - val_accuracy: 0.9500
Epoch 60/100
466/466 [==============================] - 0s 98us/step - loss: 0.0224 - accuracy: 0.9957 - val_loss: 0.1064 - val_accuracy: 0.9550
Epoch 61/100
466/466 [==============================] - 0s 89us/step - loss: 0.0222 - accuracy: 0.9957 - val_loss: 0.1206 - val_accuracy: 0.9550
Epoch 62/100
466/466 [==============================] - 0s 91us/step - loss: 0.0200 - accuracy: 0.9979 - val_loss: 0.1216 - val_accuracy: 0.9500
Epoch 63/100
466/466 [==============================] - 0s 89us/step - loss: 0.0191 - accuracy: 1.0000 - val_loss: 0.1262 - val_

In [72]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

200/200 [==============================] - 0s 75us/step
over-sampling test accuracy: 96.00%


In [77]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [78]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [79]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 466 samples, validate on 200 samples
Epoch 1/100
466/466 [==============================] - 0s 487us/step - loss: 3.8944 - accuracy: 0.5172 - val_loss: 0.7981 - val_accuracy: 0.7050
Epoch 2/100
466/466 [==============================] - 0s 131us/step - loss: 3.5549 - accuracy: 0.6266 - val_loss: 0.7777 - val_accuracy: 0.7900
Epoch 3/100
466/466 [==============================] - 0s 125us/step - loss: 3.5349 - accuracy: 0.6588 - val_loss: 0.8695 - val_accuracy: 0.7900
Epoch 4/100
466/466 [==============================] - 0s 125us/step - loss: 2.9211 - accuracy: 0.6824 - val_loss: 0.7185 - val_accuracy: 0.7900
Epoch 5/100
466/466 [==============================] - 0s 109us/step - loss: 2.9296 - accuracy: 0.6824 - val_loss: 0.7694 - val_accuracy: 0.7950
Epoch 6/100
466/466 [==============================] - 0s 102us/step - loss: 3.0221 - accuracy: 0.6567 - val_loss: 0.9313 - val_accuracy: 0.8050
Epoch 7/100
466/466 [==============================] - 0s 106us/step - loss: 3.0198 

Epoch 57/100
466/466 [==============================] - 0s 103us/step - loss: 1.4782 - accuracy: 0.7682 - val_loss: 0.7694 - val_accuracy: 0.8400
Epoch 58/100
466/466 [==============================] - 0s 91us/step - loss: 1.3176 - accuracy: 0.7639 - val_loss: 0.8712 - val_accuracy: 0.8700
Epoch 59/100
466/466 [==============================] - 0s 98us/step - loss: 1.4128 - accuracy: 0.7747 - val_loss: 0.9857 - val_accuracy: 0.8450
Epoch 60/100
466/466 [==============================] - 0s 87us/step - loss: 1.2499 - accuracy: 0.7361 - val_loss: 0.8174 - val_accuracy: 0.8750
Epoch 61/100
466/466 [==============================] - 0s 87us/step - loss: 1.6516 - accuracy: 0.7339 - val_loss: 1.0288 - val_accuracy: 0.8550
Epoch 62/100
466/466 [==============================] - 0s 92us/step - loss: 1.7086 - accuracy: 0.7403 - val_loss: 0.8008 - val_accuracy: 0.8900
Epoch 63/100
466/466 [==============================] - 0s 88us/step - loss: 1.3710 - accuracy: 0.7575 - val_loss: 0.9323 - val_a

In [80]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

200/200 [==============================] - 0s 105us/step
over-sampling test accuracy: 92.50%


In [81]:
############## Random Forest ##############

In [82]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [83]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.00%


In [84]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.96808511 1.         0.94623656 0.93548387 0.93548387]
0.9570578814916495
